In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

service = Service(ChromeDriverManager().install())

options = Options()
# options.add_argument("--headless")
options.add_argument("--window-size=1920x1080")
options.add_argument("--lang=ko_KR")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")

driver = webdriver.Chrome(service=service, options=options)

url = "https://tickets.interpark.com/contents/genre/concert"
driver.get(url)
time.sleep(2)

try :
    hot_button = driver.find_element(By.XPATH, "//button[contains(text(), '요즘 HOT')]")
    print(hot_button)
    hot_button.click()
    time.sleep(2)
    
except Exception as e :
    print("요즘 HOT 버튼 클릭 실패 : ", e)

scroll_pause = 1
last_height = driver.execute_script("return document.body.scrollHeight")

for _ in range(5) :
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height :
        break
    last_height = new_height

titles = driver.find_elements(By.CSS_SELECTOR, ".TicketItem_goodsName__Ju76j")
dates = driver.find_elements(By.CSS_SELECTOR, ".TicketItem_playDate__5ePr2")
places = driver.find_elements(By.CSS_SELECTOR, ".TicketItem_placeName__ls_9C")

data = []

for t, d, p in zip(titles, dates, places) :
    data.append({
        "title":t.text.strip(),
        "date":d.text.strip(),
        "place":p.text.strip()
    })

driver.quit()

df = pd.DataFrame(data)
df.head()

<selenium.webdriver.remote.webelement.WebElement (session="38de95e0fb09acae3a84b40b2b463501", element="f.BAA5F0E4DA905117D868B83531E01B93.d.D7051C5F840262ADBC027360B58BCB06.e.46")>


,title,date,place
0,LOVE CHIPS FESTIVAL 2025,2025.09.06 - 2025.09.07,상상플랫폼 야외광장
1,2025 이승철 콘서트 〈오케스트락2〉,2025.08.02 - 2025.09.14,"인천,당진,광주"
2,"2025 타일러, 더 크리에이터(Tyler, The Creator)내한공연",2025.09.13 ~ 2025.09.14,킨텍스 제2전시장 10홀
3,［데뷔 60주년 기념공연］2025 남진 전국투어 콘서트,2025.05.18 -,전국 각 지역
4,2025 김창옥 토크콘서트 시즌5,2025.04.19 -,전국 각 지역


In [7]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pymysql

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='akzpxld12#4',
    db='interparkmkt',
    charset='utf8mb4'
)

cursor = conn.cursor()

for _, row in df.iterrows() :
    sql = """
    INSERT INTO performances (title, date_range, place)
    VALUES (%s, %s, %s)
    """
    cursor.execute(sql, (row["title"], row["date"], row["place"]))

conn.commit()
conn.close()